In [1]:
!pip install autogluon-tabular -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.3/487.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 81.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encoders 2.7.0 requires scikit-learn<1.6.0,>=1.0.0, but you have scikit-learn 1.7.2 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
sklearn-compat 0.1.3 requires scikit-learn<1.7,>=1.2, but you have scikit-learn 1.7.2 which is incompatible.


In [2]:
import pandas as pd
import catboost as ctb
import numpy as np
import autogluon.tabular as agt

In [3]:
train = pd.read_parquet('/kaggle/input/vseros-avito-stage1/train_dataset.parquet')
test = pd.read_parquet('/kaggle/input/vseros-avito-stage1/test_dataset.parquet')
sub = pd.read_csv('/kaggle/input/vseros-avito-stage1/sample_submission.csv')

In [4]:
cat_cols = ['body_type', 'drive_type', 'engine_type', 'pts', 'diski', 'steering_wheel', 'usilitel_rul', 'electropodemniki', 'fary', 'salon', 'upravlenie_klimatom']
text_cols = ["aktivnaya_bezopasnost_mult", "audiosistema_mult", "shini_i_diski_mult", "electroprivod_mult", "fary_mult", "multimedia_navigacia_mult", "obogrev_mult", "pamyat_nastroek_mult", "podushki_bezopasnosti_mult", "pomosh_pri_vozhdenii_mult", "protivoygonnaya_sistema_mult", "salon_mult", "upravlenie_klimatom_mult", 'equipment', 'color']

In [5]:
multiple_features = [
    'aktivnaya_bezopasnost_mult', 'audiosistema_mult', 'shini_i_diski_mult',
    'electroprivod_mult', 'fary_mult', 'multimedia_navigacia_mult',
    'obogrev_mult', 'pamyat_nastroek_mult', 'podushki_bezopasnosti_mult',
    'pomosh_pri_vozhdenii_mult', 'protivoygonnaya_sistema_mult', 'salon_mult',
    'upravlenie_klimatom_mult'
]

In [6]:
from sklearn.preprocessing import MultiLabelBinarizer

In [7]:
def clean_array_data(arr):
    """Очищает numpy массив от None значений"""
    result = []
    for item in arr:
        if item is None:
            result.append([])
        elif hasattr(item, '__iter__') and not isinstance(item, str):
            result.append([x for x in item if x is not None])
        else:
            result.append([item] if item is not None else [])
    return result

new_features = []
test_features = []

for col in multiple_features:

    clean_data = clean_array_data(train[col])
    
    mlb = MultiLabelBinarizer()
    encoded = mlb.fit_transform(clean_data)
    
    new_features.append(pd.DataFrame(encoded, columns = col + "_" + mlb.classes_))
    test_features.append(pd.DataFrame(mlb.transform(clean_array_data(test[col])), columns = col + "_" + mlb.classes_))

In [8]:
train = pd.concat([train, pd.concat(new_features, axis=1)], axis=1)
test = pd.concat([test, pd.concat(test_features, axis=1)], axis=1)

In [9]:
# train[text_cols] = train[text_cols].astype(str)
# test[text_cols] = test[text_cols].astype(str)

In [10]:
def combine_text_features(row):
    text_parts = []
    for col in text_cols:
        if pd.notna(row[col]) and row[col] != "":
            if row[col] != '[None]':
                text_parts.append(f"{col} : {row[col]}")
    if len(text_parts) == 0:
        return "None"
    return "\n".join(text_parts)

def prepare_dataset(df):
    df = df.copy()
    # df[[f'image_{i}' for i in range(4)]] = [[f"{idx}_{i}" for i in range(4)] for idx in df['ID']]
    df['audiosistema'] = df['audiosistema'].fillna('0').str[0].astype(int)
    df['owners_count'] = df['owners_count'].replace({'> 3': 4}).astype(int)
    df['crashes_count'] = df['crashes_count'].replace({'3+': 4}).fillna('-1').astype(int)
    # df['text'] = df.apply(combine_text_features, axis=1)
    return df

train_df = prepare_dataset(train)
test_df = prepare_dataset(test)

In [11]:
# features = ['body_type', 'drive_type', 'engine_type', 'doors_number', 'pts',
#        'audiosistema', 'diski', 'electropodemniki', 'fary', 'salon',
#        'upravlenie_klimatom', 'usilitel_rul', 'steering_wheel',
#        'crashes_count', 'owners_count', 'mileage', 'latitude', 'longitude',
#        # , 'image_0', 'image_1', 
#             # 'image_2', 'image_3',
#             'text']
features = train_df.drop(columns=multiple_features + ['ID', 'price_TARGET']).columns.values
target = 'price_TARGET'

In [12]:
train_df[cat_cols + ['equipment', 'color']] = train_df[cat_cols + ['equipment', 'color']].fillna('None')
test_df[cat_cols + ['equipment', 'color']] = test_df[cat_cols + ['equipment', 'color']].fillna('None')

In [13]:
class MdAPEMetric:
    def is_max_optimal(self):
        return False  # Минимизируем ошибку (меньше = лучше)
    
    def evaluate(self, approxes, target, weight):
        """
        approxes - список предсказаний
        target - настоящие значения  
        weight - веса (необязательно)
        """
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])
        
        y_pred = np.array(approxes[0])
        y_true = np.array(target)
        
        # Обработка нулевых и очень маленьких значений
        epsilon = 1e-10
        mask = np.abs(y_true) > epsilon
        
        if np.sum(mask) == 0:
            return 1.0, 1  # Если все значения нулевые, возвращаем максимальную ошибку
        
        y_true_filtered = y_true[mask]
        y_pred_filtered = y_pred[mask]
        
        # Вычисляем абсолютные процентные ошибки (без умножения на 100)
        ape = np.abs((y_true_filtered - y_pred_filtered) / y_true_filtered)
        mdape_value = np.median(ape)  # Диапазон [0, ∞), но обычно [0, 1]
        
        return mdape_value, 1
    
    def get_final_error(self, error, weight):
        return error

In [14]:
# tr_embs = np.load('/kaggle/input/vseros-avito-effnet-b3-embeddings/train_embeddings_b3.npy')
# test_embs = np.load('/kaggle/input/vseros-avito-effnet-b3-embeddings/test_embeddings_b3.npy')

In [15]:
# tr_embs = np.load('/kaggle/input/vseros-avito-gluonmm-baseline/tr_embeddings_b2.npy')
# test_embs = np.load('/kaggle/input/vseros-avito-gluonmm-baseline/test_embeddings_b2.npy')

In [16]:
# tr_embs = np.load('/kaggle/input/vseros-avito-gluonmm-baseline-vit-base/tr_embeddings_vit-base.npy')
# test_embs = np.load('/kaggle/input/vseros-avito-gluonmm-baseline-vit-base/test_embeddings_vit-base.npy')

In [17]:
# tr_embs = np.load('/kaggle/input/vseros-avito-gluonmm-baseline-vit-large/tr_embeddings_vit-large.npy')
# test_embs = np.load('/kaggle/input/vseros-avito-gluonmm-baseline-vit-large/test_embeddings_vit-large.npy')

In [18]:
# from sklearn.decomposition import PCA

# pca = PCA(n_components=256)
# train_df[[f'vit_embs_{i}' for i in range(pca.n_components)]] = pca.fit_transform(tr_embs)
# test_df[[f'vit_embs_{i}' for i in range(pca.n_components)]] = pca.transform(test_embs)
# features = np.hstack([features, [f'vit_embs_{i}' for i in range(pca.n_components)]])

In [19]:
# tr_embs = np.load('/kaggle/input/vseros-avito-gluonmm-baseline-convnext-large/tr_embeddings_convnext-large.npy')
# test_embs = np.load('/kaggle/input/vseros-avito-gluonmm-baseline-convnext-large/test_embeddings_convnext-large.npy')

In [20]:
# from sklearn.decomposition import PCA

# pca = PCA(n_components=256)
# train_df[[f'convnext_embs_{i}' for i in range(pca.n_components)]] = pca.fit_transform(tr_embs)
# test_df[[f'convnext_embs_{i}' for i in range(pca.n_components)]] = pca.transform(test_embs)
# # features = np.hstack([features, [f'convnext_embs_{i}' for i in range(pca.n_components)]])

In [21]:
tr_embs = np.load('/kaggle/input/vseros-avito-gluonmm-baseline-eva/tr_embeddings_eva-large.npy')
test_embs = np.load('/kaggle/input/vseros-avito-gluonmm-baseline-eva/test_embeddings_eva-large.npy')

In [22]:
from sklearn.decomposition import PCA

pca = PCA(n_components=256)
train_df[[f'eva_embs_{i}' for i in range(pca.n_components)]] = pca.fit_transform(tr_embs)
test_df[[f'eva_embs_{i}' for i in range(pca.n_components)]] = pca.transform(test_embs)
features = np.hstack([features, [f'eva_embs_{i}' for i in range(pca.n_components)]])

/tmp/ipykernel_13/3256920729.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_df[[f'eva_embs_{i}' for i in range(pca.n_components)]] = pca.fit_transform(tr_embs)
/tmp/ipykernel_13/3256920729.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_df[[f'eva_embs_{i}' for i in range(pca.n_components)]] = pca.fit_transform(tr_embs)
/tmp/ipykernel_13/3256920729.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider j

In [23]:
features = np.hstack([features, ['target']])

In [24]:
train_df['target'] = np.log1p(train_df[target] // 1000)

/tmp/ipykernel_13/4230511189.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_df['target'] = np.log1p(train_df[target] // 1000)


In [25]:
def median_ape(y_true, y_pred, epsilon=1e-10):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Проверка размеров
    if len(y_true) != len(y_pred):
        raise ValueError("y_true и y_pred должны иметь одинаковую длину")
    
    # Избегаем деления на ноль
    denominator = np.where(np.abs(y_true) < epsilon, epsilon, y_true)
    
    # Вычисляем абсолютные процентные ошибки
    ape = np.abs((y_true - y_pred) / denominator)
    
    # Возвращаем медиану
    return float(np.median(ape))

In [26]:
from autogluon.core.metrics import make_scorer

mdape = make_scorer(name='mdape',
                    score_func=median_ape,
                    optimum=0,
                    greater_is_better=False,
                    needs_pred=True,
                   )

In [27]:
train_df['car_condition_clip	car_class_clip	car_style_clip'.split()] = pd.read_csv('/kaggle/input/vseros-avito-clip-features/clip_features_train.csv')['car_condition_clip	car_class_clip	car_style_clip'.split()].values
test_df['car_condition_clip	car_class_clip	car_style_clip'.split()] = pd.read_csv('/kaggle/input/vseros-avito-clip-features/clip_features_test.csv')['car_condition_clip	car_class_clip	car_style_clip'.split()].values

/tmp/ipykernel_13/2036979633.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_df['car_condition_clip	car_class_clip	car_style_clip'.split()] = pd.read_csv('/kaggle/input/vseros-avito-clip-features/clip_features_train.csv')['car_condition_clip	car_class_clip	car_style_clip'.split()].values
/tmp/ipykernel_13/2036979633.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_df['car_condition_clip	car_class_clip	car_style_clip'.split()] = pd.read_csv('/kaggle/input/vseros-avito-clip-features/clip_features_train.csv')['ca

In [28]:
train['car_condition_clip	car_class_clip	car_style_clip'.split()] = pd.read_csv('/kaggle/input/vseros-avito-clip-features/clip_features_train.csv')['car_condition_clip	car_class_clip	car_style_clip'.split()].values

In [29]:
# features = np.hstack([features, 'car_condition_clip	car_class_clip	car_style_clip'.split()])

In [30]:
from sklearn.model_selection import KFold
cv = KFold(n_splits=5)
models = []
scores = []

for i, (tr_idx, val_idx) in enumerate(cv.split(train_df)):
    tr, val = train_df.iloc[tr_idx], train_df.iloc[val_idx]

    predictor = agt.TabularPredictor(label='target', eval_metric=mdape).fit(tr[features], tuning_data=val[features], time_limit=3600, hyperparameters={
    	'NN_TORCH': [{}],
    	'GBM': [{'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}, {}, {'learning_rate': 0.03, 'num_leaves': 128, 'feature_fraction': 0.9, 'min_data_in_leaf': 3, 'ag_args': {'name_suffix': 'Large', 'priority': 0, 'hyperparameter_tune_kwargs': None}}],
    	# 'CAT': [{}],
    	'XGB': [{}],
    	'FASTAI': [{}],
    	# 'RF': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
    	# 'XT': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
    })
    # pred = np.expm1(predictor.predict(val, as_pandas=False)) * 1000
    pred = np.expm1(predictor.predict(val, as_pandas=False)) * 1000
    scores.append(median_ape(val[target].values, pred))
    models.append(predictor)
    print("-" * 10)
    print(f"Fold #{i + 1}: {median_ape(val[target].values, pred):.5f}")
    print("-" * 10)

print("-" * 10)
print("CV Score")
print("-" * 10)
print(f"{np.mean(scores):.5f} ± {np.std(scores):.5f}")

No path specified. Models will be saved in: "AutogluonModels/ag-20251004_040309"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.11.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       29.03 GB / 31.35 GB (92.6%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='extreme' : New in v1.4: Massively better than 'best' on datasets <30000 samples by using new models meta-learned on https://tabarena.ai: TabPFNv2, TabICL, Mitra, and TabM. Absolute best accuracy. Requires a GPU. Recommended 64 GB CPU memory and 32+ GB GPU mem

----------
Fold #1: 0.12099
----------


Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "/kaggle/working/AutogluonModels/ag-20251004_041106"
Train Data Rows:    56000
Train Data Columns: 334
Tuning Data Rows:    14000
Tuning Data Columns: 334
Label Column:       target
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (10.168156850678836, 2.3978952727983707, 6.09365, 1.00123)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during Predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression', 'quantile'])
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    29007.25 MB
	Train Data (Original)  Memory Usage: 189.33 MB (0.7% of available memory)
	Inferring d

----------
Fold #2: 0.12133
----------


Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "/kaggle/working/AutogluonModels/ag-20251004_041831"
Train Data Rows:    56000
Train Data Columns: 334
Tuning Data Rows:    14000
Tuning Data Columns: 334
Label Column:       target
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (10.168156850678836, 2.3978952727983707, 6.09553, 1.00048)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during Predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression', 'quantile'])
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    29056.36 MB
	Train Data (Original)  Memory Usage: 189.33 MB (0.7% of available memory)
	Inferring d

...

----------
Fold #4: 0.11919
----------


Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "/kaggle/working/AutogluonModels/ag-20251004_043145"
Train Data Rows:    56000
Train Data Columns: 334
Tuning Data Rows:    14000
Tuning Data Columns: 334
Label Column:       target
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (10.168156850678836, 2.3978952727983707, 6.09765, 1.00331)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during Predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression', 'quantile'])
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    29022.15 MB
	Train Data (Original)  Memory Usage: 189.33 MB (0.7% of available memory)
	Inferring d

[1000]	valid_set's l2: 0.0545151	valid_set's mdape: -0.0205063


	-0.0204	 = Validation score   (-mdape)
	51.11s	 = Training   runtime
	0.84s	 = Validation runtime
Fitting model: LightGBM ... Training model for up to 3544.85s of the 3544.85s of remaining time.
	Fitting with cpus=2, gpus=0, mem=0.4/28.2 GB
	-0.0206	 = Validation score   (-mdape)
	18.67s	 = Training   runtime
	0.19s	 = Validation runtime
Fitting model: NeuralNetFastAI ... Training model for up to 3525.97s of the 3525.97s of remaining time.
	Fitting with cpus=2, gpus=0, mem=0.6/28.2 GB
Metric mdape is not supported by this model - using mean_squared_error instead
No improvement since epoch 6: early stopping
	-0.0226	 = Validation score   (-mdape)
	88.62s	 = Training   runtime
	0.33s	 = Validation runtime
Fitting model: XGBoost ... Training model for up to 3436.97s of the 3436.96s of remaining time.
	Fitting with cpus=2, gpus=0
	-0.0206	 = Validation score   (-mdape)
	94.85s	 = Training   runtime
	0.59s	 = Validation runtime
Fitting model: NeuralNetTorch ... Training model for up to 334

----------
Fold #5: 0.12313
----------
----------
CV Score
----------
0.12114 ± 0.00125


In [31]:
# predictor = agt.TabularPredictor(label='target', eval_metric=mdape).fit(train_df[features], time_limit=3600, hyperparameters={
# 	'NN_TORCH': [{}],
# 	'GBM': [{'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}, {}, {'learning_rate': 0.03, 'num_leaves': 128, 'feature_fraction': 0.9, 'min_data_in_leaf': 3, 'ag_args': {'name_suffix': 'Large', 'priority': 0, 'hyperparameter_tune_kwargs': None}}],
# 	# 'CAT': [{}],
# 	'XGB': [{}],
# 	'FASTAI': [{}],
# 	# 'RF': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
# 	# 'XT': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
# })

In [32]:
# sub['target'] = np.expm1(predictor.predict(test_df, as_pandas=False)) * 1000
# sub.to_csv('avito_gluon_div1000_effnetb3_256_log_clip_features-more.csv', index=False)

In [33]:
# preds = []
# for i in range(cv.n_splits):
#     preds.append(models[i].predict(test_df, as_pandas=False))
# sub['target'] = np.expm1(np.mean(preds, 0)) * 1000
# sub.to_csv('avito_gluon_div1000_effnetb3_256_log_clip_features_5folds_mean-exp.csv', index=False)

In [34]:
preds = []
for i in range(cv.n_splits):
    preds.append(np.expm1(models[i].predict(test_df, as_pandas=False)) * 1000)
sub['target'] = np.mean(preds, 0)
sub.to_csv('avito_gluon_div1000_eva-large-35per_256_log_clip_features_5folds-mdae.csv', index=False)